<a href="https://colab.research.google.com/github/lanjunq/Airbnb-Property-Price-Recommendation/blob/master/Airbnb_Price_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 519 Final Project - Airbnb Price Recommendation

# 1. Import Data and Preprocessing

## Import Raw Data

In [0]:
from google.colab import drive
import glob

import pandas as pd
import numpy as np

from joblib import dump, load

import matplotlib.pyplot as plt


In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shared\ drives/CIS\ 519\ Project\ -\ Airbnb\ Pricing\ Recommendation/data


In [0]:
# Build a raw data dictionary
import glob
import os

data_sources = glob.glob('*')
data_dict = dict()
for source in data_sources:
  data_city = os.listdir(source)
  data_dict[source] = dict()

  for city in data_city:
    data_file = os.listdir(source + '/' + city)
    data_dict[source][city] = data_file

# Visualize all data files
for source in data_dict.keys():
  print('Source: {}'.format(source))
  for city in data_dict[source].keys():
    print('\tCity: {}'.format(city))
    for raw_data in data_dict[source][city]:
      print('\t\tFile: \t {}'.format(raw_data))


In [0]:
# source = 'Inside_Airbnb'
# city_ny = 'New_York'
# city_la = "Los_Angeles"
# f = 'listings.csv.gz'
# ny_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, 'New_York', f) )
# la_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, 'Los_Angeles', f) )
# # ny_reviews_df.iloc[:, 40:].head(5)

## Preprocessing

In [0]:
ny_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, city_ny, f) )
la_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, city_la, f) )

feature_selection = ["id", "experiences_offered", "transit", "host_response_time" , "host_response_rate" ,  
                     "host_acceptance_rate",  "host_is_superhost", "host_neighbourhood", "host_total_listings_count",  
                     "host_verifications",  "host_identity_verified" , "neighbourhood_cleansed" , "neighbourhood_group_cleansed" , 
                     "zipcode" , "latitude", "longitude",  "property_type", "room_type", "accommodates", "bathrooms", "bedrooms", 
                     "beds", "bed_type", "amenities", "square_feet", "security_deposit" , "cleaning_fee", "guests_included" , "extra_people" , 
                     "minimum_nights" , "number_of_reviews",  "number_of_reviews_ltm", "review_scores_rating" , 
                     "review_scores_accuracy",  "review_scores_cleanliness",  "review_scores_checkin" , 
                     "review_scores_communication",  "review_scores_location" , "review_scores_value", "requires_license" , 
                     "instant_bookable" ,  "cancellation_policy", "price" ]



> Processing
*   experiences_offered: none - 0, otherwise 1
*   transits: nan - 0, otherwise 1
*   host_response_time: 5 catogories, nan - most frequent (haven't hot-encoding)
*   host_response_rate, host_acceptance_rate: convert str to float, kept nan for now
*   host_id_superhost: nan - most frequent ('f'), 'f' - 0, 't' - 1
*   host_neighbourhood: dropped
*   host_total_listings_count: nan - 1 (most freq)
*   host_verifications: dropped replaced by host_verifications_count: based on host_verifications: missing value - 0, otherwise: len
*   host_identity_verified: nan - most frequent ('f'), 'f' - 0, 't' - 1
*   neighbourhood_cleansed, neighbourhood_group_cleansed: kept & haven't done anything
*   zipcode: missing value - fre, converted to 5-digit float
*   latitude, longitude: kept as float
*   property_type, bed_type, room_type: kept as str for now
*   accommadates: no missing value, kept as number
*   bathrooms, bedrooms,beds: replace nan with frequent num
*   amenities
*   amenities: dropped replaced by amenities_count: len of amenities
*   square_feet: missing - freq (or avg? could try)
*   security_deposit, cleaning_fee, extra_people - missing value - 0, then  get rid of ',' and '\$' convert to float
*   guests_included: no missing, kept as int
*   minimum_nights, number_of_reviews, number_of_reviews_ltm: no missing, kept as int
*   review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value: missing replaced with 0 (?should this be avg? or 0?)
*   requires_license, instant_bookable: 1 't'  0 'f'
*   cancellation_policy: 6 categories
*   price: get rid of ',' and '\$' convert to float








In [0]:
X_ny = ny_reviews_df[feature_selection]
print(type(X_ny))
print(X_ny.shape)
X_ny.head()

In [0]:
print(X_ny.isnull().sum(axis=0)/X_ny.shape[0])
for i in range(43):
#   if(i == 2 or i == 14 or i == 15 or i ==17 or i == 21):
    print("the number of unique value of the column " + str(feature_selection[i]) +" is: "+ str(len(X_ny[X_ny.columns[i]].unique())))
    # print("unique valur is: ")
    # print(ny_X[ny_X.columns[i]].unique())
    #print("the most freq value is: " + str(X_train[X_train.columns[i]].mode()) + "\n")
    # print("data type is: " + str(type(X.columns[i])))
    # print("the most freq value is: " + str(ny_X[ny_X.columns[i]].value_counts().idxmax()) + "\n")
#   else:
    # X_train[col_names[i]] = X_train[col_names[i]].astype(float)

In [0]:
# feature type conversion
X_ny["experiences_offered"] = X_ny["experiences_offered"].apply(lambda x : 0 if(x == 'none') else 1)
X_ny['transit'] = X_ny['transit'].apply(lambda x : 0 if(type(x) is float) else 1)
X_ny['host_response_time'] = X_ny['host_response_time'].apply(lambda x : X_ny["host_response_time"].value_counts().idxmax() if(type(x) is float) else x)
X_ny['host_response_rate'] = X_ny['host_response_rate'].apply(lambda x : int(x.split('%')[0])/100 if(type(x) is not float) else x)
X_ny['host_acceptance_rate'] = X_ny['host_acceptance_rate'].apply(lambda x : int(x.split('%')[0])/100 if(type(x) is not float) else x)
X_ny['host_is_superhost'] = X_ny['host_is_superhost'].apply(lambda x : 'f' if(type(x) is float) else x)
X_ny['host_is_superhost'] = X_ny['host_is_superhost'].apply(lambda x : 0 if(x == 'f') else 1)
X_ny['host_total_listings_count'] =X_ny['host_total_listings_count'].apply(lambda x : 1.0 if(np.isnan(x)) else x)
X_ny['host_verifications_count'] =X_ny['host_verifications'].apply(lambda x : 0 if(x == '[]') else x)
X_ny['host_verifications_count'] = X_ny['host_verifications'].apply(lambda x : x if(type(x) is int) else len(x.split(',')))
X_ny['host_identity_verified'] = X_ny['host_identity_verified'].apply(lambda x : 'f' if(type(x) is float) else x)
X_ny['host_identity_verified'] = X_ny['host_identity_verified'].apply(lambda x : 0 if(x == 'f') else 1)
X_ny['zipcode'] = X_ny['zipcode'].apply(lambda x : X_ny["zipcode"].value_counts().idxmax() if(type(x) is float or x==' ')  else x)
X_ny['zipcode'] = X_ny['zipcode'].apply(lambda x : x[3:] if('NY' in x or ' ' in x) else x)
X_ny['zipcode'] = X_ny['zipcode'].apply(lambda x : x[:5] if(len(x)>5) else x)
X_ny['zipcode'] = X_ny['zipcode'].apply(lambda x : float(x))
# replace nan with frequent num for "bathrooms","bedrooms","beds" - don't know if the value means the number of bath or the type?
X_ny['bathrooms'] = X_ny['bathrooms'].apply(lambda x : X_ny["bathrooms"].value_counts().idxmax() if(np.isnan(x))  else x)
X_ny['bedrooms'] = X_ny['bedrooms'].apply(lambda x : X_ny["bedrooms"].value_counts().idxmax() if(np.isnan(x))  else x)
X_ny['beds'] = X_ny['beds'].apply(lambda x : X_ny["beds"].value_counts().idxmax() if(np.isnan(x))  else x)
X_ny['amenities_count'] = X_ny['amenities'].apply(lambda x : x if(type(x) is int) else len(x.split(',')))
X_ny["square_feet"] = X_ny["square_feet"].apply(lambda x : X_ny["square_feet"].value_counts().idxmax() if(np.isnan(x))  else x)
X_ny["security_deposit"]=  X_ny["security_deposit"].apply(lambda x : '$0' if(type(x) is float)  else x)
X_ny["security_deposit"]=  X_ny["security_deposit"].apply(lambda x : x.replace(",",""))
X_ny["security_deposit"]=  X_ny["security_deposit"].apply(lambda x: float(x[1:]))
X_ny["cleaning_fee"]=  X_ny["cleaning_fee"].apply(lambda x : '$0' if(type(x) is float)  else x)
X_ny["cleaning_fee"]=  X_ny["cleaning_fee"].apply(lambda x : x.replace(",",""))
X_ny["cleaning_fee"]= X_ny["cleaning_fee"].apply(lambda x: float(x[1:]))
X_ny["extra_people"]=  X_ny["extra_people"].apply(lambda x : '$0' if(type(x) is float)  else x)
X_ny["extra_people"]=  X_ny["extra_people"].apply(lambda x : x.replace(",",""))
X_ny["extra_people"]=  X_ny["extra_people"].apply(lambda x: float(x[1:]))
X_ny['minimum_nights'] = X_ny['minimum_nights'].apply(lambda x : 1 if(np.isnan(x))  else x)
X_ny['number_of_reviews'] = X_ny['number_of_reviews'].apply(lambda x : 1 if(np.isnan(x))  else x)
X_ny['number_of_reviews_ltm'] = X_ny['number_of_reviews_ltm'].apply(lambda x : 1 if(np.isnan(x))  else x)
X_ny["review_scores_rating"] = X_ny["review_scores_rating"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_accuracy"] = X_ny["review_scores_accuracy"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_cleanliness"] = X_ny["review_scores_cleanliness"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_checkin"] = X_ny["review_scores_checkin"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_communication"] = X_ny["review_scores_communication"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_location"] = X_ny["review_scores_location"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["review_scores_value"] = X_ny["review_scores_value"].apply(lambda x: 0.0 if(np.isnan(x)) else x)
X_ny["requires_license"] = X_ny["requires_license"].apply(lambda x: 1 if(x == 't') else 0)
X_ny["instant_bookable"] = X_ny["requires_license"].apply(lambda x: 1 if(x == 't') else 0)
X_ny["price"] = X_ny["price"].apply(lambda x : '$0' if(type(x) is float)  else x)
X_ny["price"] = X_ny["price"].apply(lambda x : x.replace(",",""))
X_ny["price"] = X_ny["price"].apply(lambda x: float(x[1:]))

In [0]:
#replace nan with avg for: host_response_rate, host_acceptance_rate
avg_response_rate = X_ny["host_response_rate"].sum()/(X_ny["host_response_rate"].shape[0] - X_ny["host_response_rate"].isna().sum())
print(avg_response_rate)
X_ny["host_response_rate"] = X_ny["host_response_rate"].apply(lambda x: avg_response_rate if(np.isnan(x)) else x)
#
avg_acceptance_rate= X_ny["host_acceptance_rate"].sum()/(X_ny["host_acceptance_rate"].shape[0] - X_ny["host_acceptance_rate"].isna().sum())
X_ny["host_acceptance_rate"] = X_ny["host_acceptance_rate"].apply(lambda x: avg_acceptance_rate if(np.isnan(x)) else x)

In [0]:
X_ny = X_ny.drop(["host_neighbourhood"], axis=1)
X_ny = X_ny.drop(["amenities"], axis=1)
X_ny = X_ny.drop(["host_verifications"], axis=1)

In [0]:
# Save df as .csv
X_ny.to_csv( '{}/{}/{}'.format(source, city_ny, "cleaned_ny.csv") ,index=False)

# Read df from .csv
X_ny = pd.read_csv( '{}/{}/{}'.format(source, city_ny, "cleaned_ny.csv"))
print(X_ny.shape, '\n', X_ny.dtypes)
X_ny.head(5)

In [0]:
print(X_ny.isnull().sum(axis=0)/X_ny.shape[0])

In [0]:
print("\nData Type:")
print("data shape: ("+ str(X_ny.shape[0]) +" , " +str(X_ny.shape[1]) + ")")
print(X_ny.dtypes)

In [0]:
print("unique value of room_type is: ")
print(X_ny["room_type"].unique())
print('\n')

print("unique value of bed_type is: ")
print(X_ny["bed_type"].unique())
print('\n')

print("unique value of property_type is: ")
print(X_ny["property_type"].unique())
print('\n')

print("unique value of cancellation_policy is: ")
print(X_ny["cancellation_policy"].unique())

Preprocessing for LA Data

In [0]:
X_la = la_reviews_df[feature_selection]
print(type(X_la))
print(X_la.shape)
X_la.head()

One-hot encoding

In [0]:
X_ny = pd.read_csv( '{}/{}/{}'.format(source, city_ny, "cleaned_ny.csv"))
X_la = pd.read_csv( '{}/{}/{}'.format(source, city_la, "cleaned_la.csv"))

#onehot encoding
#drop column

In [0]:
# #data for visualization
# ny = X_ny[["latitude", "longitude","price"]]
# la = X_la[["latitude", "longitude","price"]]

# # display(ny.head(5))
# # display(la.head(5))

# ny.to_csv( '{}/{}/{}'.format(source, city_ny, "ny_price.csv") ,index=False)
# la.to_csv( '{}/{}/{}'.format(source, city_la, "la_price.csv") ,index=False)

In [0]:
categoorical_ny = X_ny[["host_response_time","neighbourhood_group_cleansed", "property_type","bed_type","room_type","cancellation_policy"]].reset_index(drop=True)
categoorical_la = X_la[["host_response_time","neighbourhood_group_cleansed", "property_type","bed_type","room_type","cancellation_policy"]].reset_index(drop=True)


In [0]:
X_ny = pd.get_dummies(X_ny, columns=["host_response_time"])
X_la = pd.get_dummies(X_la, columns=["host_response_time"])

X_ny = pd.get_dummies(X_ny, columns=["neighbourhood_group_cleansed"])
X_la = pd.get_dummies(X_la, columns=["neighbourhood_group_cleansed"])

X_ny = pd.get_dummies(X_ny, columns=["property_type"])
X_la = pd.get_dummies(X_la, columns=["property_type"])

X_ny = pd.get_dummies(X_ny, columns=["room_type"])
X_la = pd.get_dummies(X_la, columns=["room_type"])

X_ny = pd.get_dummies(X_ny, columns=["bed_type"])
X_la = pd.get_dummies(X_la, columns=["bed_type"])

X_ny = pd.get_dummies(X_ny, columns=["cancellation_policy"])
X_la = pd.get_dummies(X_la, columns=["cancellation_policy"])

In [0]:
X_ny.to_csv( '{}/{}/{}'.format(source, city_ny, "cleaned_ny_encoding.csv") ,index=False)
X_la.to_csv( '{}/{}/{}'.format(source, city_la, "cleaned_la_encoding.csv") ,index=False)

## Train-Test Split
Test is going to be 1/10 of the whole datasets.

In [0]:
X_ny = pd.read_csv( '{}/{}/{}'.format(source, city_ny, "cleaned_ny_encoding.csv"))
X_la = pd.read_csv( '{}/{}/{}'.format(source, city_la, "cleaned_la_encoding.csv"))

In [0]:
from sklearn.model_selection import train_test_split

ny_train, ny_test = train_test_split(X_ny, test_size=0.1, random_state=42, shuffle = True)
la_train, la_test = train_test_split(X_la, test_size=0.1, random_state=42, shuffle = True)

## Standardization and Normalization

In [0]:
ny_columns = list(ny_train.columns)
la_columns = list(la_train.columns)
ny_columns.remove("id")
la_columns.remove("id")
ny_columns.remove("price")
la_columns.remove("price")
category_ny = ny_columns[33:]
category_la = la_columns[33:]
category_ny.append("id")
category_ny.append("price")
category_la.append("id")
category_la.append("price")
print(category_ny)
print(category_la)

In [0]:

y_ny_train = ny_train[category_ny].reset_index(drop=True)
X_ny_train = ny_train.drop(category_ny, axis=1).reset_index(drop=True)



In [0]:
y_la_train = la_train[category_la].reset_index(drop=True)
X_la_train = la_train.drop(category_la, axis=1).reset_index(drop=True)

y_ny_test = ny_test[category_ny].reset_index(drop=True)
X_ny_test = ny_test.drop(category_ny, axis=1).reset_index(drop=True)

y_la_test = la_test[category_la].reset_index(drop=True)
X_la_test = la_test.drop(category_la, axis=1).reset_index(drop=True)

In [0]:
from sklearn.preprocessing import StandardScaler

scaler_ny = StandardScaler()
X_ny_train = scaler_ny.fit_transform(X_ny_train)
X_ny_test = scaler_ny.transform(X_ny_test)

scaler_la = StandardScaler()
X_la_train = scaler_la.fit_transform(X_la_train)
X_la_test = scaler_la.transform(X_la_test)

In [0]:
from sklearn.preprocessing import Normalizer

nor_ny = Normalizer()
X_ny_train = nor_ny.fit_transform(X_ny_train)
X_ny_test = nor_ny.transform(X_ny_test)

nor_la = Normalizer()
X_la_train = nor_la.fit_transform(X_la_train)
X_la_test = nor_la.transform(X_la_test)

In [0]:
ny_train = pd.DataFrame(data=X_ny_train,  columns = ny_columns[:33])
for c_column in category_ny:
    ny_train[c_column] = y_ny_train[c_column]


la_train = pd.DataFrame(data=X_la_train,  columns=la_columns[:33])
for c_column in category_la:
    la_train[c_column] = y_la_train[c_column]


ny_test= pd.DataFrame(data=X_ny_test,  columns=ny_columns[:33])
for c_column in category_ny:
    ny_test[c_column] = y_ny_test[c_column]


la_test= pd.DataFrame(data=X_la_test,  columns=la_columns[:33])
for c_column in category_la:
    la_test[c_column] = y_la_test[c_column]

In [0]:
ny_train.to_csv( '{}/{}/{}'.format(source, city_ny, "train_ny.csv") ,index=False)
ny_test.to_csv( '{}/{}/{}'.format(source, city_ny, "test_ny.csv") ,index=False)

la_train.to_csv( '{}/{}/{}'.format(source, city_la, "train_la.csv") ,index=False)
la_test.to_csv( '{}/{}/{}'.format(source, city_la, "test_la.csv") ,index=False)



In [0]:
ny_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, city_ny, f) )
la_reviews_df = pd.read_csv( '{}/{}/{}'.format(source, city_la, f) )

feature_selection = ["street" , "neighbourhood_cleansed", "neighbourhood_group_cleansed", 
                     "city", "state",  "country", "zipcode", "latitude", 'longitude', "price" ]

# 2. Helper Functions for Training and Testing

## Correlation Matrix

In [0]:
#
import seaborn as sn
temp = pd.concat([X_ny_train, y_ny_train], axis=1)
# display(temp.head(10))

# display(y_ny_train)

corrMatrix = temp.corr()
# print(corrMatrix.iloc[:,-1].sort_values())
for ind in corrMatrix.iloc[:,-1].sort_values().index.tolist():
  # ind = corrMatrix.index.tolist()[i]
  # print(ind)
  print('{:50s}: \t {}'.format(ind, corrMatrix.loc[ind, 'price']))

sn.heatmap(np.ravel(corrMatrix.iloc[:,-1]).reshape(len(corrMatrix), 1), annot=True)
# sn.heatmap(corrMatrix, annot=True)

plt.show()

In [0]:
features_to_drop = ['latitude', 'longitude', 'zipcode']
# sub_X_la_train = X_la_train.drop(features_to_drop, axis=1)
# display(sub_X_la_train.head(10))

## Testing Process Helper Functions

In [0]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
import graphviz 
from sklearn import metrics

### Scatter Plot

In [0]:
def scatter_plot(y_test, y_test_pred, y_train, y_train_pred, model_name):
    '''
    A scatterplot of predicted vs. actual values 
    with a line representing where predicted = actual values
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(str('Predicted vs. actual values - '+model_name), fontsize=14, y=1)
    plt.subplots_adjust(top=0.93, wspace=0)
    
    ax1.scatter(y_test, y_test_pred, s=2, alpha=0.7)
    ax1.plot(list(range(0,450)), list(range(50,500)), color='blue', linestyle='dashdot') 
    ax1.plot(list(range(0,500)), list(range(0,500)), color='black', linestyle='--')
    ax1.plot(list(range(50,500)), list(range(0,450)), color='blue', linestyle='dashdot')
    
    ax1.set_title('Test set')
    ax1.set_xlabel('Actual values')
    ax1.set_ylabel('Predicted values')
    
    ax2.scatter(y_train, y_train_pred, s=2, alpha=0.7)
    ax2.plot(list(range(0,450)), list(range(50,500)), color='blue', linestyle='dashdot')    
    ax2.plot(list(range(0,500)), list(range(0,500)), color='black', linestyle='--')
    ax2.plot(list(range(50,500)), list(range(0,450)), color='blue', linestyle='dashdot')
    ax2.set_title('Train set')
    ax2.set_xlabel('Actual values')
    ax2.set_ylabel('')
    ax2.set_yticklabels(labels='')
    
    plt.show()

### Learning Curve

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)


    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    plt.legend(loc="best")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.suptitle(title)


    return plt

In [0]:
ny_price = ny_reviews_df[feature_selection]
la_price = la_reviews_df[feature_selection]

In [0]:
ny_price["city"] = "New York"
ny_price["state"] = "New York"
ny_price["country"] = "United States"

print(ny_price.isnull().sum(axis=0)/ny_price.shape[0])
display(ny_price)

In [0]:
la_price["city"] = "Los Angeles"
la_price["state"] = "California"
la_price["country"] = "United States"

print(la_price.isnull().sum(axis=0)/la_price.shape[0])
display(la_price)

In [0]:
ny_price.to_csv( '{}/{}/{}'.format(source, city_ny, "ny_price.csv") ,index=False)
la_price.to_csv( '{}/{}/{}'.format(source, city_la, "la_price.csv") ,index=False)

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error

## Training Process Helper Functions

In [0]:
def train_model(model, X_train, y_train):
  
  %time model.fit(X_train, y_train)

  r2_score_cv = cross_val_score(model, X_train, y_train, scoring='r2', cv=3)
  print('average r2 score: {},\t cross-validation: {}'.format(np.mean(r2_score_cv), r2_score_cv))

  mse_score_cv = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=3)
  print('average mse score: {},\t cross-validation: {}'.format(np.mean(mse_score_cv), mse_score_cv))
  
  return model

# 3. Training & Testing

## Decision Tree Regressor

In [0]:
%%time
# Train with randomized grid search


''' class sklearn.tree.DecisionTreeRegressor(
  criterion='mse', 
  splitter='best', 
  max_depth=None, 
  min_samples_split=2, 
  min_samples_leaf=1, 
  min_weight_fraction_leaf=0.0, 
  max_features=None, 
  random_state=None, 
  max_leaf_nodes=None, 
  min_impurity_decrease=0.0, 
  min_impurity_split=None, 
  presort='deprecated', 
  ccp_alpha=0.0)
'''

parameters = {  'max_depth' : list(range(10,200,10)),
                'criterion' : ['mae'],
                'min_samples_leaf' : [3, 8, 20, 30, 50, 70, 100],
                'min_samples_split' : [3, 8, 20, 50, 80, 100]
              }

clf = RandomizedSearchCV(tree.DecisionTreeRegressor(), parameters, n_iter=40, cv=5)
clf.fit(X_la_train.drop(features_to_drop, axis=1), y_la_train)

results_df = pd.DataFrame(clf.cv_results_)
display(results_df)

print('best score: ', clf.best_score_)
print('best parameters: ', clf.best_params_)


In [0]:
# Test Decision Tree

best = clf.best_estimator_
print(best)

print('Decision Tree Test Scores:')

print('Training: ')
print('r2 score:  \t{}'.format(r2_score(best.predict(X_la_train), y_la_train)))
print('mse score: \t{}'.format(mean_squared_error(best.predict(X_la_train), y_la_train)))
print('mae score: \t{}'.format(mean_absolute_error(best.predict(X_la_train), y_la_train)))

print('Testing: ')
print('r2 score:  \t{}'.format(r2_score(best.predict(X_la_test), y_la_test)))
print('mse score: \t{}'.format(mean_squared_error(best.predict(X_la_test), y_la_test)))
print('mae score: \t{}'.format(mean_absolute_error(best.predict(X_la_test), y_la_test)))

In [0]:
 # visualize ccp_alpha effect on post-pruning 
import matplotlib.pyplot as plt

path = best.cost_complexity_pruning_path(X_ny_train, y_ny_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

## Random Forest Regressor

In [0]:
print('''Random Forest''')

'''
class sklearn.ensemble.RandomForestRegressor(
  n_estimators=100, 
  criterion='mse', 
  max_depth=None, 
  min_samples_split=2, 
  min_samples_leaf=1, 
  min_weight_fraction_leaf=0.0, 
  max_features='auto', 
  max_leaf_nodes=None, 
  min_impurity_decrease=0.0, 
  min_impurity_split=None, 
  bootstrap=True, 
  oob_score=False, 
  n_jobs=None, 
  random_state=None, 
  verbose=0, 
  warm_start=False, 
  ccp_alpha=0.0, 
  max_samples=None)
'''

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, max_depth=3, min_samples_split=10)
model_randomforest_ny = train_model(model, X_ny_train, y_ny_train)


In [0]:
%%time
# Train with randomized grid search
from sklearn.ensemble import RandomForestRegressor

parameters = {  'n_estimators': [100, 150, 200],
                'max_depth' : list(range(2,20,3)),
                'min_samples_leaf' : [1, 2, 3, 5, 8, 10, 15, 20],
                'min_samples_split' : [1, 2, 3, 5, 8, 10, 15, 20],
                'ccp_alpha' : [0, 0.2, 0.5, 1, 10, 30, 60, 100]
              }

clf = RandomizedSearchCV(RandomForestRegressor(), parameters, n_iter=20, cv=2)
clf.fit(X_la_train, np.ravel(y_la_train))

results_df = pd.DataFrame(clf.cv_results_)
display(results_df)
# display(results_df[['param_max_depth', 'mean_test_score']])

print(clf.best_score_)
print(clf.best_params_)


In [0]:
# Test Random Forest

best = clf.best_estimator_
print(best)

print('Random Forest Test Scores:')

print('Training: ')
print('r2 score:  \t{}'.format(r2_score(best.predict(X_la_train), y_la_train)))
print('mse score: \t{}'.format(mean_squared_error(best.predict(X_la_train), y_la_train)))
print('mae score: \t{}'.format(mean_absolute_error(best.predict(X_la_train), y_la_train)))

print('Testing: ')
print('r2 score:  \t{}'.format(r2_score(best.predict(X_la_test), y_la_test)))
print('mse score: \t{}'.format(mean_squared_error(best.predict(X_la_test), y_la_test)))
print('mae score: \t{}'.format(mean_absolute_error(best.predict(X_la_test), y_la_test)))

## Linear Regression

In [0]:
from sklearn import linear_model
model = linear_model.Ridge(alpha=.5)
model = model.fit(X_ny_train, y_ny_train)
model_linear_ny = train_model(model, X_ny_train, y_ny_train)

In [0]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
def test_linear(model, X_test, y_test, X_train, y_train, title):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    mae = mean_absolute_error(y_test, y_test_pred)
    print("mse score is: " + str(mse))
    print("r2 score is: " + str(r2))
    print("mae score is: " + str(mae))
  
    scatter_plot(y_test, y_test_pred, y_train, y_train_pred,title)
    return (mse, r2)

In [0]:
test_linear(model_linear_ny, X_ny_test,y_ny_test, X_ny_train,y_ny_train,"Linear")

In [0]:
title = "Learning Curves (Linear Regression)"
plt = plot_learning_curve(model_linear_ny, title, X_ny_train, y_ny_train, axes=None, ylim=None, cv= 5)
plt.savefig("Linear_learning_curve.png")

## Neural Network

In [0]:
from sklearn.metrics import mean_absolute_error
def test_NN(model,skip_epochs, X_test, y_test, X_train, y_train, title):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    mae = mean_absolute_error(y_test, y_test_pred)
    print("mse score is: " + str(mse))
    print("r2 score is: " + str(r2))
    print("mae score is: " + str(mae))
   
    # Line graph of losses
    model_results = model.history.history
    plt.plot(list(range((skip_epochs+1),len(model_results['loss'])+1)), model_results['loss'][skip_epochs:], label='Train',color='red')
    plt.plot(list(range((skip_epochs+1),len(model_results['val_loss'])+1)), model_results['val_loss'][skip_epochs:], label='Test', color='green')
    plt.legend()
    plt.title('Learning Curves(Neural Network)', fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("MSE Loss")
    plt.show()

    scatter_plot(y_test, y_test_pred, y_train, y_train_pred, title)   
    return (mse, r2)

In [0]:
from keras import models, layers, optimizers, regularizers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
# Building the model
nn_ny = models.Sequential()
nn_ny.add(layers.Dropout(0.2, input_shape=(X_ny_train.shape[1],)))
nn_ny.add(layers.Dense(128, activation='relu'))
nn_ny.add(layers.Dropout(0.2))
nn_ny.add(layers.Dense(256, activation='relu'))
nn_ny.add(layers.Dropout(0.2))
nn_ny.add(layers.Dense(512, activation='relu'))
nn_ny.add(layers.Dropout(0.2))
nn_ny.add(layers.Dense(1, activation='linear'))

# Compiling the model
nn_ny.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Model summary
print(nn_ny.summary())

In [0]:
import time
# Training the model
nn_ny_start = time.time()

nn_ny_history = nn_ny.fit(X_ny_train,
                  y_ny_train,
                  epochs=200,
                  batch_size=200,
                  validation_split = 0.1)

nn_ny_end = time.time()

print(f"Time taken to run: {round((nn_ny_end - nn_ny_start)/60,1)} minutes")

In [0]:
test_NN(nn_ny,20, X_ny_test,y_ny_test, X_ny_train,y_ny_train, "NN")

## KNN Regressor


In [0]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error

def train_KNN(model, X, y):
    model = model.fit(X, y)
    print("training r2: " + str(model.score(X,y)))
    return model

def train_test_KNN(model, X_test, y_test, X_train, y_train, model_name):
    model = train_KNN(model, X_train, y_train)
    y_train_pred = model.predict(X_train)
    train_mse = mean_squared_error(y_train, y_train_pred)
    print("training mse: " + str(train_mse))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    print("training mae: " + str(train_mae))

    y_test_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred)
    r2 = r2_score(y_test, y_test_pred)
    mae = mean_absolute_error(y_test, y_test_pred)
    print("mse score is: " + str(mse))
    print("r2 score is: " + str(r2))
    print("mae score is: " + str(mae))

    scatter_plot(y_test, y_test_pred, y_train, y_train_pred, model_name)
    return (mse, r2, mae)


In [0]:
model_ny = KNeighborsRegressor( n_neighbors=25, p = 1)
train_test_KNN(model_ny, subdataX_test_ny, y_ny_test, subdataX_train_ny, y_ny_train, "KNN")